函数由两部分组成：代码对象（func.\_\_code__）持有字节码和指令元数据，负责执行；函数对象则为上下文提供调用实例，并管理所需的状态数据。

In [6]:
import dis

def test(x, y=10):
    x += y
    print(x,y)
dis.dis(test)

  4           0 LOAD_FAST                0 (x)
              2 LOAD_FAST                1 (y)
              4 INPLACE_ADD
              6 STORE_FAST               0 (x)

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                0 (x)
             12 LOAD_FAST                1 (y)
             14 CALL_FUNCTION            2
             16 POP_TOP
             18 LOAD_CONST               0 (None)
             20 RETURN_VALUE


In [8]:
dis.dis(test.__code__.co_code) # 没有元数据符号的反汇编结果

          0 LOAD_FAST                0 (0)
          2 LOAD_FAST                1 (1)
          4 INPLACE_ADD
          6 STORE_FAST               0 (0)
          8 LOAD_GLOBAL              0 (0)
         10 LOAD_FAST                0 (0)
         12 LOAD_FAST                1 (1)
         14 CALL_FUNCTION            2
         16 POP_TOP
         18 LOAD_CONST               0 (0)
         20 RETURN_VALUE


In [9]:
test.__defaults__

(10,)

In [14]:
test.__defaults__ = (1234,)
test(1)

1235 1234


In [13]:
test.abn = 'sd'
test.__dict__

{'abn': 'sd'}

def实际上是运行期指令，以代码对象为参数，创建函数实例，并在当前上下文中与指定的名字相关联。